# Training workflow

In [1]:
import torch
import pandas as pd
import numpy as np
import os
import copy

from IPython.display import Video

## Constants

In [2]:
CUDA = torch.cuda.is_available()
RANDOM_SEED = 1
BATCH_SIZE = 128
LEARNING_RATE = 1e-4
NUM_EPOCHS = 15

print(f"CUDA: {CUDA}")

CUDA: True


In [3]:
np.random.seed(RANDOM_SEED)

## Load Data

Extracted features, scores, metadata, etc.

In [4]:
data = pd.read_csv("training_set/scores.csv").set_index("video_id")

In [5]:
Video(data.iloc[22]["video_url"])

In [6]:
def load_C3D_features(video_ids, path ="training_set/Features/C3D/"):
    features = []
    for video_id in video_ids:
        filename = f"{path}{f'{video_id}'.zfill(5)}.mp4.csv"
        features.append(np.loadtxt(filename, delimiter=","))
    return features

In [7]:
data["c3d"] = load_C3D_features(data.index)

## Data Prep

Building datasets

potentially weighting samples based on annotations?



In [8]:
data.head()

,video_url,ann_1,ann_2,part_1_scores,part_2_scores,c3d
video_id,,,,,,
8,https://mtc.cdn.vine.co/r/videos_h264high/9EB0...,5,3,1.00,1.00,"[0.0, 0.0, 0.0, 0.0, 0.1541, 0.3501, 0.1796, 0..."
26,https://mtc.cdn.vine.co/r/videos_h264high/A8B3...,6,4,1.00,0.75,"[0.6788, 0.2429, 0.0, 0.0, 0.8737, 0.0, 0.0, 0..."
33,https://mtc.cdn.vine.co/r/videos/267829AEFA128...,7,4,0.71,0.75,"[0.511, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
46,https://mtc.cdn.vine.co/r/videos_h264high/B974...,6,4,1.00,0.25,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.363..."
64,https://mtc.cdn.vine.co/r/videos_h264high/C4D6...,6,7,0.83,0.43,"[0.4755, 0.0, 0.0153, 0.0, 0.0, 0.1077, 0.0, 0..."


In [9]:
def split_training(data: pd.DataFrame, shuffle=True, split=0.8):
    ids = np.random.permutation(list(data.index)) if shuffle else list(data.index)
    split_index = int(len(ids) * split)
    return data.loc[ids[:split_index]], data.loc[ids[split_index:]]

In [10]:
train, valid = split_training(data)
print(len(train), len(valid))

472 118


In [11]:
def build_matrixes(data: pd.DataFrame, targets, features, dtype="double"):
    feature_matrix = [np.concatenate(list(row_features), axis=None) 
                      for row_features in zip(*[list(data[feature]) for feature in features])]
    target_matrix = [np.concatenate(row_targets, axis = None) 
                     for row_targets in zip(*[list(data[target]) for target in targets])]
    return np.array(feature_matrix, dtype=dtype), np.array(target_matrix, dtype=dtype)
        

In [12]:
targets = ["part_1_scores", "part_2_scores"]
features = ["c3d"]

features_train, targets_train = build_matrixes(train, targets = targets, features = features)
features_valid, targets_valid = build_matrixes(valid, targets = targets, features = features)

print(len(features_train), len(features_valid), len(targets_train), len(targets_valid))

472 118 472 118


In [13]:
class FlatDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [14]:
data_train = FlatDataset(features_train, targets_train)
data_valid = FlatDataset(features_valid, targets_valid)
print(len(data_train), len(data_valid))

472 118


## Model setup

In [15]:
input_dim = len(features_train[0])
output_dim = len(targets_train[0])
print(input_dim, output_dim)

4096 2


In [16]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim = 1024):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x
        

In [17]:
model = TwoLayerNet(input_dim, output_dim)

## Training setup

In [18]:
num_epochs = NUM_EPOCHS

In [19]:
batch_size = BATCH_SIZE

In [20]:
learning_rate = LEARNING_RATE

In [21]:
loss_fn = torch.nn.MSELoss(reduction="sum")

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [23]:
device = torch.device("cuda") if CUDA else torch.device("cpu")
model = model.double().to(device)

In [24]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma = 0.1)

In [25]:
dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True)
dataloader_valid = torch.utils.data.DataLoader(data_valid, batch_size=1)

## Train model

In [26]:
def train(model, dataloader_train, dataloader_valid, loss_fn, optimizer, scheduler, device):
    
    for is_training in [True, False]: # Epoch is a training followed by validation 
        
        model.train() if is_training else model.eval()
        
        running_loss = 0
        for features, targets in (dataloader_train if is_training else dataloader_valid):
            features = features.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(is_training):
                outputs = model(features)
                _, preds = torch.max(outputs, 1)
                loss = loss_fn(outputs, targets)
                if is_training:
                    loss.backward()
                    optimizer.step()               
            running_loss += loss.item()
        
        if is_training:
            scheduler.step()
        
        if is_training:
            train_loss = running_loss / len(dataloader_train.dataset)
        else:
            valid_loss = running_loss / len(dataloader_valid.dataset)
    
    return train_loss, valid_loss

In [27]:
best_valid_loss = 9999999
best_model_state_dict = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    print(f"--------------- Epoch {epoch} ----------------")
    train_loss, val_loss = train(model = model,
                                 dataloader_train = dataloader_train,
                                 dataloader_valid = dataloader_valid,
                                 loss_fn = loss_fn,
                                 optimizer = optimizer,
                                 scheduler = scheduler,
                                 device = device)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        best_model_state_dict = copy.deepcopy(model.state_dict())
        print("New Best Validiation Loss!!!", val_loss)
    
    print("Training Loss:", train_loss)
    print("Validation Loss:", val_loss)
print("\n\nFINISHED TRAINING")
print(f"Best validation lost: {best_valid_loss}")

--------------- Epoch 0 ----------------
New Best Validiation Loss!!! 0.7178543238019993
Training Loss: 0.7310521936900505
Validation Loss: 0.7178543238019993
--------------- Epoch 1 ----------------
New Best Validiation Loss!!! 0.11523285561869924
Training Loss: 0.3913191823458899
Validation Loss: 0.11523285561869924
--------------- Epoch 2 ----------------
Training Loss: 0.17817531035959744
Validation Loss: 0.250456654742675
--------------- Epoch 3 ----------------
Training Loss: 0.23853697597961349
Validation Loss: 0.22833728613120075
--------------- Epoch 4 ----------------
Training Loss: 0.20820969153931795
Validation Loss: 0.1870486117016761
--------------- Epoch 5 ----------------
Training Loss: 0.16448670123396747
Validation Loss: 0.14556989838783918
--------------- Epoch 6 ----------------
Training Loss: 0.1359182247466396
Validation Loss: 0.14176831089265057
--------------- Epoch 7 ----------------
Training Loss: 0.13189525021429163
Validation Loss: 0.13800831978660894
------

In [28]:
best_model = model.load_state_dict(best_model_state_dict)

## Test

Spearman's rank correlation, ROC curves, etc